

 <h1>
Finetuning of Llama3-8b on math problem

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Model will output True if the solution is correct, and False otherwise.

## **Step 1: Install Necessary Libraries**

We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU.


In [ ]:
# %%capture
# Since xformer and latest version of unsloth will lead to error, we just use unsloth(2025.10.10) and do not use xformer then.
!pip install unsloth==2025.10.10

  Using cached unsloth-2025.10.10-py3-none-any.whl.metadata (61 kB)
  Using cached unsloth_zoo-2025.10.13-py3-none-any.whl.metadata (32 kB)
  Using cached tyro-0.9.35-py3-none-any.whl.metadata (12 kB)
  Using cached xformers-0.0.32.post2-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (1.1 kB)
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.56.2-py3-none-any.whl.metadata (40 kB)
  Using cached trl-0.23.0-py3-none-any.whl.metadata (11 kB)
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
  Using cached torchao-0.14.1-cp310-abi3-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (19 kB)
  Using cached cut_cross_entropy-25.1.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached msgspec-0.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.9 kB)
  Using cached shtab-1.7.2-py3-none-any.whl.metadata (

## **Step 2: Load the Model and Tokenizer**

#Option1:
We'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch

max_seq_length = 1500  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

#Option2:
Some of the finetuned models do not performed well, but we do not want to repeated the training from the start. So we load the saved checkpoints and continue the training to save time.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel
import torch
from google.colab import drive
drive.mount('/content/drive')
max_seq_length = 1500  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint11"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

model.train()
print(f"Model and tokenizer loaded from: {save_path}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


🦥 Unsloth Zoo will now patch everything to make training faster!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
==((====))==  Unsloth 2025.10.10: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.318 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.10.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model and tokenizer loaded from: /content/drive/MyDrive/llama3_8b_math_verifier_checkpoint11


## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. So we just use the subset for training and validation. The range of data used can be check in the report.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset

# Loading
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")
shuffled_dataset = full_dataset.shuffle(seed=42)
subset = shuffled_dataset.select(range(200000))

# Splitting
train_size = int(0.95 * len(subset))
train_dataset = subset.select(range(train_size))
validation_dataset = subset.select(range(train_size, len(subset)))

In [ ]:
# Prompting
training_prompt = """You are a mathematics expert. Carefully read the following question and its proposed solution.
Your task is to determine whether the provided solution correctly solves the question.
First, analyze the reasoning in the solution, then decide if the final answer is logically and mathematically correct.
Respond with “True” or “False” and explain your reasoning briefly.

Question:
{question}

Solution:
{solution}

Your reasoning:
<Think step by step and explain why the solution is correct or not before giving your final answer.>

Final Answer:
{output}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]

    texts = [
        training_prompt.format(
            question=question,
            solution=str(solution) if solution is not None else "",
            output=str(output) if output is not None else ""
        ) + EOS_TOKEN
        for question, solution, output in zip(questions, solutions, outputs)
    ]
    return {"text": texts}

formatted_train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
    batch_size=1000,
    num_proc=2,
)


Map (num_proc=2):   0%|          | 0/190000 [00:00<?, ? examples/s]

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 256,  # Larger rank to explain well
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    lora_alpha = 512,
    lora_dropout = 0,

    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth 2025.10.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

The parameters can be checked in report.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
FastLanguageModel.for_inference(model)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=True,

    args=TrainingArguments(
        # epoch and batch
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        num_train_epochs = 1,

        # learning rate
        learning_rate=5e-5,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",

        # optimizer
        optim="adamw_8bit",
        weight_decay=0.1,
        max_grad_norm=1.0,
        adam_beta2=0.98,

        # precision
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),

        # log and save
        logging_steps=100,
        output_dir = "outputs",
        report_to="none",
        seed=42,
        remove_unused_columns=True,
        group_by_length = False,
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/190000 [00:00<?, ? examples/s]

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
# avoid ram overflow in colab
torch.cuda.empty_cache()

In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 190,000 | Num Epochs = 1 | Total steps = 11,875
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 2 x 1) = 16
 "-____-"     Trainable parameters = 671,088,640 of 8,701,349,888 (7.71% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
100,1.030500
200,0.616500
300,0.595800
400,0.585800
500,0.581700
600,0.573500
700,0.572400
800,0.558600
900,0.550800
1000,0.551400


TrainOutput(global_step=11875, training_loss=0.4044070795962685, metrics={'train_runtime': 31792.1988, 'train_samples_per_second': 5.976, 'train_steps_per_second': 0.374, 'total_flos': 5.220684984553243e+18, 'train_loss': 0.4044070795962685, 'epoch': 1.0})



---


## **Step 6: Inference and Evaluation**

For large subset we chosen, the validation set is still too large when inference. So for validation, we just some samples in the subset.

In [ ]:
from tqdm import tqdm
import torch

FastLanguageModel.for_inference(model)

val_prompt = """You are a mathematics expert. Carefully read the following question and its proposed solution.
Your task is to determine whether the provided solution correctly solves the question.
First, analyze the reasoning in the solution, then decide if the final answer is logically and mathematically correct.
Respond with only 'True' or 'False' at the end.

Question:
{}

Solution:
{}

Your reasoning:
<Think step by step and explain why the solution is correct or not before giving your final answer.>

Final Answer:
"""

def parse_prediction(response):
    if "Final Answer:" in response:
        output_part = response.split("Final Answer:")[-1].strip()
    else:
        output_part = response
    return 'true' in output_part.lower()

val_samples = validation_dataset.select(range(1000))
correct = 0
batch_size = 32

for i in tqdm(range(0, len(val_samples), batch_size)):
    batch = val_samples[i:i+batch_size]
    prompts = [val_prompt.format(q, str(s)) for q, s in zip(batch["question"], batch["solution"])]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=max_seq_length).to("cuda")

    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=64, do_sample=False, use_cache=True)

    responses = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    preds = [parse_prediction(r) for r in responses]
    correct += sum(p == l for p, l in zip(preds, batch["is_correct"]))

accuracy = correct / len(val_samples)
print(f"ACC: {accuracy:.4f} ({correct}/{len(val_samples)})")

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
FastLanguageModel.for_inference(model)
# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

inference_prompt = """You are a mathematics expert. Carefully read the following question and its proposed solution.
Your task is to determine whether the provided solution correctly solves the question.
First, analyze the reasoning in the solution, then decide if the final answer is logically and mathematically correct.
Respond with only 'True' or 'False' at the end.

Question:
{}

Solution:
{}

Your reasoning:
<Think step by step and explain why the solution is correct or not before giving your final answer.>

Final Answer:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Final Answer:"
    output_part = response_text.split("Final Answer:")[-1]
    # Check if "True" appears (case-insensitive)
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)  # 🔼 稍微增大 max_new_tokens 以容纳 reasoning
    response_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\n✅ Submission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

100%|██████████| 10000/10000 [35:11<00:00,  4.74it/s]


✅ Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: /content/drive/MyDrive/llama3_8b_math_verifier_checkpoint11
